In [5]:
#########Importing libraries##########
import numpy as np
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from PIL import  Image
import itertools
import warnings
warnings.filterwarnings("ignore")
import io

############Visualization#############
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls
import plotly.figure_factory as ff
######################################

In [6]:
########Bring in the DATA!###########

telcom = pd.read_csv("db/telemarker_db.csv")
telcom.head()
print(telcom.dtypes)
#######Data Cleaning Process#########

#Replacing spaces with null values in total charges column
telcom['TotalCharges'] = telcom["TotalCharges"].replace(" ",np.nan)
print("Percentage of null: " + str(round(telcom['TotalCharges'].isnull().sum()/telcom['TotalCharges'].count()*100,4))+"%")

#Dropping null values from total charges column which contain .16% missing data
telcom = telcom[telcom["TotalCharges"].notnull()]
telcom = telcom.reset_index()[telcom.columns]

#convert to float type
telcom["TotalCharges"] = telcom["TotalCharges"].astype(float)
telcom["MonthlyCharges"] = telcom["MonthlyCharges"].astype(float)

#replace 'No phone service' to No
telcom["MultipleLines"] = telcom["MultipleLines"].replace("No phone service","No")

#replace 'No internet service' to No for the following columns
replace_cols = [ 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection',
                'TechSupport','StreamingTV', 'StreamingMovies']
for i in replace_cols : 
    telcom[i]  = telcom[i].replace({'No internet service' : 'No'})

#Change types of columns to boolean
YesNoColumns = ["Partner","Dependents","PhoneService","MultipleLines","OnlineSecurity",
                "OnlineBackup","DeviceProtection","TechSupport","StreamingTV","StreamingMovies",
                "PaperlessBilling","Churn"]
for i in YesNoColumns :
    telcom[i]  = telcom[i].replace({'Yes':1,'No':0})
BoolColumns = YesNoColumns.copy()
BoolColumns.append("SeniorCitizen")
telcom[BoolColumns] = telcom[BoolColumns].astype('bool')
telcom
#Separating churn and non churn customers
# churn     = telcom[telcom["Churn"] == "Yes"]
# not_churn = telcom[telcom["Churn"] == "No"]

#     #Separating catagorical and numerical columns
# Id_col     = ['customerID']
# target_col = ["Churn"]
# cat_cols   = telcom.nunique()[telcom.nunique() < 6].keys().tolist()
# cat_cols   = [x for x in cat_cols if x not in target_col]
# num_cols   = [x for x in telcom.columns if x not in cat_cols + target_col + Id_col]

customerID           object
gender               object
City                 object
State                object
Phone                object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
PaperlessBilling     object
PaymentMethod        object
MonthlyCharges      float64
TotalCharges         object
Churn                object
Name                 object
Address              object
dtype: object
Percentage of null: 0.1564%


,customerID,gender,City,State,Phone,SeniorCitizen,Partner,Dependents,tenure,PhoneService,...,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn,Name,Address
0,7590-VHVEG,Female,Sujzojpab,CA,(461) 230-8635,False,True,False,1,False,...,False,False,Month-to-month,True,Electronic check,29.85,29.85,False,Margaret Gregory,1927 Gusta Way
1,5575-GNVDE,Male,Bozuto,WY,(500) 288-7672,False,False,False,34,True,...,False,False,One year,False,Mailed check,56.95,1889.50,False,Allie Valdez,525 Ahev Grove
2,3668-QPYBK,Male,Cefzizof,NY,(368) 757-6896,False,False,False,2,True,...,False,False,Month-to-month,True,Mailed check,53.85,108.15,True,Bertha Padilla,985 Utomez Extension
3,7795-CFOCW,Male,Zavictej,NV,(865) 723-2538,False,False,False,45,False,...,False,False,One year,False,Bank transfer (automatic),42.30,1840.75,False,Ian Perez,1030 Gagicu Avenue
4,9237-HQITU,Female,Socgimo,HI,(206) 668-9076,False,False,False,2,True,...,False,False,Month-to-month,True,Electronic check,70.70,151.65,True,Gilbert Murray,1905 Ukiij Path
5,9305-CDSKC,Female,Japamber,VA,(854) 862-8556,False,False,False,8,True,...,True,True,Month-to-month,True,Electronic check,99.65,820.50,True,Hunter Powers,1019 Tunzu Pike
6,1452-KIOVK,Male,Mejtoof,SD,(880) 500-3425,False,False,True,22,True,...,True,False,Month-to-month,True,Credit card (automatic),89.10,1949.40,False,Sallie Wheeler,1988 Vutiko Ridge
7,6713-OKOMC,Female,Bohbozlo,WI,(318) 983-2359,False,False,False,10,False,...,False,False,Month-to-month,False,Mailed check,29.75,301.90,False,Charles Sandoval,1471 Kapad Trail
8,7892-POOKP,Female,Urtahfor,NC,(451) 608-9497,False,True,False,28,True,...,True,True,Month-to-month,True,Electronic check,104.80,3046.05,True,Birdie Bryant,1620 Vagih Point
9,6388-TABGU,Male,Zuzusojo,MI,(659) 896-7415,False,False,True,62,True,...,False,False,One year,False,Bank transfer (automatic),56.15,3487.95,False,Elmer McKinney,1187 Obase Pass


In [7]:
#labels
lab = telcom["Churn"].value_counts().keys().tolist()
#values
val = telcom["Churn"].value_counts().values.tolist()

trace = go.Pie(labels = lab ,
               values = val ,
               marker = dict(colors =  [ 'royalblue' ,'lime'],
                             line = dict(color = "white",
                                         width =  1.3)
                            ),
               rotation = 90,
               hoverinfo = "label+value+text",
               hole = .5
              )
layout = go.Layout(dict(title = "Customer attrition in data",
                        plot_bgcolor  = "rgb(243,243,243)",
                        paper_bgcolor = "rgb(243,243,243)",
                       )
                  )

data = [trace]
fig = go.Figure(data = data,layout = layout)
py.iplot(fig)